# Simulating Steller Orbits Around Sagittarius A

Arian Andalib, Nate Gu, Elias Taira

## Introduction

While Sagittarius A has been observed many times and have been confirmed to be the center of our galaxy since as early as the 1980's, it wasn't until the 2000s that this "star" was confirmed to be a black hole. Even more recently, we have been able to observe the orbits of stars very close to the black hole. Then, using Kepler's Laws to map out the orbital path of the stars, obtain an accurate reading of what the mass of this black hole is.

![](./Galactic_centre_orbits.svg.png)

After a careful analysis of the astronomical data, it was found that the bleck hole weighs around $3.6^{+0.2}_{-0.4}*10^6 M_\odot$ 

The actual data for these orbits can be found below from the Wikepidia page on Sag A

In [1]:
import pandas as pd
sagAdat = pd.read_excel("SagittariusA_data.xlsx")
sagAdat

,id1,id2,a,Δa,e,Δe,i (°),Δi,Ω (°),ΔΩ,...,Δω,Tp (yr),ΔTp,P (yr),ΔP,Kmag,q (AU),Δq,v (%c),Δv
0,S1,S0-1,0.5950,0.0240,0.5560,0.0180,119.14,0.21,342.04,0.32,...,1.40,2001.800,0.150,166.0,5.8,14.70,2160.7,6.7,0.55,0.03
1,S2,S0-2,0.1251,0.0001,0.8843,0.0001,133.91,0.05,228.07,0.04,...,0.04,2018.379,0.001,16.1,0.0,13.95,118.4,0.2,2.56,0.00
2,S8,S0-4,0.4047,0.0014,0.8031,0.0075,74.37,0.30,315.43,0.19,...,0.41,1983.640,0.240,92.9,0.4,14.50,651.7,22.5,1.07,0.01
3,S12,S0-19,0.2987,0.0018,0.8883,0.0017,33.56,0.49,230.10,1.80,...,1.50,1995.590,0.040,58.9,0.2,15.50,272.9,2.0,1.69,0.01
4,S13,S0-20,0.2641,0.0016,0.4250,0.0023,24.70,0.48,74.50,1.70,...,2.40,2004.860,0.040,49.0,0.1,15.80,1242.0,2.4,0.69,0.01
5,S14,S0-16,0.2863,0.0036,0.9761,0.0037,100.59,0.87,226.38,0.64,...,0.87,2000.120,0.060,55.3,0.5,15.70,56.0,3.8,3.83,0.06
6,S62,NaN,0.0905,0.0001,0.9760,0.0020,72.76,4.58,122.61,0.57,...,0.40,2003.330,0.010,9.9,0.0,16.10,16.4,1.5,7.03,0.04
7,S4714,NaN,0.1020,0.0120,0.9850,0.0110,127.70,0.28,129.28,0.63,...,0.08,2017.290,0.020,12.0,0.3,17.70,12.6,9.3,8.00,3.00


In this experiment, we seek to understand how exactly these orbital pathways were reconstructed using this observational data, and then reconstruct this system ourselves, in both 2 and 3 dimensions, using only the initial positions and velocities of the stars as well as the mass of Sag A.

To do this, we will employ the use of the Velocity Verlet method, an iterative method designed to solve various kinds of differential equations. In the scope of this project we will use it to solve for the positions and velocities of the nearby stars orbiting Sag A given their initial position, velocity and the gravitational force exerted on them by the central black hole.

The differential equation we will be solving is denoted by: $$a(r) = \frac{GM_sr}{\textbf{r}^3} $$ where $a(r)$ represents the acceleration of the star (derived from the gravitational force), $G$ represents the gravitational constant ($6.67*10^{-11} m^3 kg^{-1} s^{-2}$), $M_s$ represents the mass of Sag A, **r** represents the distance to Sag A in meters and, r represents the position vector of the star.

To obtain these initial conditions, we decided to use the **q (AU)** column for position and the  **v(%c)** column for velocity. **q (AU)** represents the distance of the star from the black hole at its **perhilion**, the closest point to the black hole in the star's orbital path. This distance is measured in **AU** or **Astronomical Units**, which represent the distance from Earth to the Sun (i.e., the distance from Earth to the Sun is 1 AU or ~1.5e11 m). However, for this project we will be working in SI units, so this will be converted into meters. **v(%c)** represents the speed of the star at this perhilion location. This is units of **%c** where 'c' represents the speed of light at $3*10^8$ m/s. We will also be using this in SI units as well (m/s).

## 2D Orbit Simulation

## 3D Orbit Simulation

## Conclusion